In [1]:
### The stock option class. Calculates and stores common attributes of options

import math

""" 
Stores common attributes of a stock option 
"""
class StockOption(object):
    def __init__(
        self, S0, K, r=0.05, T=1, N=2, pu=0, pd=0, 
        div=0, sigma=0, is_put=False, is_am=False):
        """
        Initialize the stock option base class.
        Defaults to European call unless specified.

        :param S0: initial stock price
        :param K: strike price
        :param r: risk-free interest rate
        :param T: time to maturity
        :param N: number of time steps
        :param pu: probability at up state
        :param pd: probability at down state
        :param div: Dividend yield
        :param is_put: True for a put option,
                False for a call option
        :param is_am: True for an American option,
                False for a European option
        """
        self.S0 = S0
        self.K = K
        self.r = r
        self.T = T
        self.N = max(1, N)
        self.STs = [] # Declare the stock prices tree

        """ Optional parameters used by derived classes """
        self.pu, self.pd = pu, pd
        self.div = div
        self.sigma = sigma
        self.is_call = not is_put
        self.is_european = not is_am

    @property
    def dt(self):
        """ Single time step, in years """
        return self.T/float(self.N)

    @property
    def df(self):
        """ The discount factor """
        return math.exp(-(self.r-self.div)*self.dt)  

In [22]:
import math
import numpy as np

""" 
Price a European or American option by the binomial tree 
"""
class BinomialTreeOption(StockOption):

    def setup_parameters(self):
        self.u = 1+self.pu  # Expected value in the up state
        self.d = 1-self.pd  # Expected value in the down state
        self.qu = (math.exp(
            (self.r-self.div)*self.dt)-self.d)/(self.u-self.d)
        self.qd = 1-self.qu

    def init_stock_price_tree(self):
        # Initialize a 2D tree at T=0
        self.STs = [np.array([self.S0])]

        # Simulate the possible stock prices path
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate(
                (prev_branches*self.u, 
                 [prev_branches[-1]*self.d]))
            self.STs.append(st) # Add nodes at each time step

    def init_payoffs_tree(self):
        if self.is_call:
            return np.maximum(0, self.STs[self.N]-self.K)
        else:
            return np.maximum(0, self.K-self.STs[self.N])

    def check_early_exercise(self, payoffs, node):
        if self.is_call:
            return np.maximum(payoffs, self.STs[node] - self.K)
        else:
            return np.maximum(payoffs, self.K - self.STs[node])

    def traverse_tree(self, payoffs):
        for i in reversed(range(self.N)):
            # The payoffs from NOT exercising the option
            payoffs = (payoffs[:-1]*self.qu + 
                       payoffs[1:]*self.qd)*self.df

            # Payoffs from exercising, for American options
            if not self.is_european:
                payoffs = self.check_early_exercise(payoffs,i)

        return payoffs

    def begin_tree_traversal(self):
        payoffs = self.init_payoffs_tree()
        return self.traverse_tree(payoffs)

    def price(self):
        """  The pricing implementation """
        self.setup_parameters()
        self.init_stock_price_tree()
        payoffs = self.begin_tree_traversal()
        return payoffs[0]

In [24]:
### To change this to the CRR model we omit params u and d and instead use params mean and var as seen below

class BinomialCRROption(BinomialTreeOption):
 def _setup_parameters_(self):
     self.u = math.exp(self.sigma * math.sqrt(self.dt))
     self.d = 1./self.u
     self.qu = (math.exp((self.r-self.div)*self.dt) -
     self.d)/(self.u-self.d)
     self.qd = 1-self.qu


In [29]:

class BinomialLROption(BinomialTreeOption):
    def setup_parameters(self):
        """
        Setup parameters for Leisen-Reimer binomial tree model
        """
        # Ensure odd number of time steps for better convergence
        odd_N = self.N if (self.N % 2 == 1) else (self.N + 1)
        
        # Calculate d1 and d2 from Black-Scholes formula
        d1 = (math.log(self.S0 / self.K) + 
              ((self.r - self.div) + (self.sigma**2) / 2.) * self.T) / \
             (self.sigma * math.sqrt(self.T))
        
        d2 = (math.log(self.S0 / self.K) + 
              ((self.r - self.div) - (self.sigma**2) / 2.) * self.T) / \
             (self.sigma * math.sqrt(self.T))
        
        # Peizer-Pratt inversion formula
        def pp_2_inversion(z, n):
            """
            Peizer-Pratt second-order inversion formula
            """
            return (0.5 + math.copysign(1, z) * 
                   math.sqrt(0.25 - 0.25 * math.exp(
                       -((z / (n + 1./3. + 0.1/(n + 1)))**2.) * (n + 1./6.))))
        
        # Calculate probabilities using Peizer-Pratt inversion
        pbar = pp_2_inversion(d1, odd_N)
        self.p = pp_2_inversion(d2, odd_N)
        
        # Calculate up and down factors
        self.u = (1 / self.df) * (pbar / self.p)
        self.d = ((1 / self.df) - self.p * self.u) / (1 - self.p)
        
        # Risk-neutral probabilities
        self.qu = self.p
        self.qd = 1 - self.p

In [30]:
class BinomialLRWithGreeks(BinomialLROption):
    def __new_stock_price_tree__(self):
        """ Create additional layer of nodes to our original stock price tree """
        self.STs = [np.array([self.S0*self.u/self.d, self.S0, self.S0*self.d/self.u])]
        for i in range(self.N):
            prev_branches = self.STs[-1]
            st = np.concatenate((prev_branches * self.u, [prev_branches[-1] * self.d]))
            self.STs.append(st)
    
    def price(self):
        self._setup_parameters_()
        self.__new_stock_price_tree__()
        # Chapter 4 [ 89 ]
        payoffs = self.__begin_tree_traversal__()
        """ Option value is now in the middle node at t=0"""
        option_value = payoffs[len(payoffs)/2]
        payoff_up = payoffs[0]
        payoff_down = payoffs[-1]
        S_up = self.STs[0][0]
        S_down = self.STs[0][-1]
        dS_up = S_up - self.S0
        dS_down = self.S0 - S_down
        """ Get delta value """
        dS = S_up - S_down
        dV = payoff_up - payoff_down
        delta = dV/dS
        """ Get gamma value """
        gamma = ((payoff_up-option_value)/dS_up - (option_value-payoff_down)/dS_down) / \
                ((self.S0+S_up)/2. - (self.S0+S_down)/2.)
        return option_value, delta, gamma

In [34]:
class TrinomialTreeOption(BinomialTreeOption):
    def setup_parameters(self):
        """ Required calculations for the trinomial tree model """
        # Up factor - stock price moves up by this multiplier
        self.u = math.exp(self.sigma * math.sqrt(2. * self.dt))
        # Down factor - reciprocal of up factor
        self.d = 1 / self.u
        # Middle factor - stock price stays the same
        self.m = 1
        
        # Risk-neutral probability for upward movement
        self.qu = ((math.exp((self.r - self.div) * self.dt / 2.) - 
                    math.exp(-self.sigma * math.sqrt(self.dt / 2.))) / 
                   (math.exp(self.sigma * math.sqrt(self.dt / 2.)) - 
                    math.exp(-self.sigma * math.sqrt(self.dt / 2.))))**2
        
        # Risk-neutral probability for downward movement
        self.qd = ((math.exp(self.sigma * math.sqrt(self.dt / 2.)) - 
                    math.exp((self.r - self.div) * self.dt / 2.)) / 
                   (math.exp(self.sigma * math.sqrt(self.dt / 2.)) - 
                    math.exp(-self.sigma * math.sqrt(self.dt / 2.))))**2.
        
        # Risk-neutral probability for no movement (middle node)
        self.qm = 1 - self.qu - self.qd
    
    def initialize_stock_price_tree(self):
        """ Initialize a 2D trinomial tree at t=0 """
        # Start with initial stock price at t=0
        self.STs = [np.array([self.S0])]
        
        # Build tree forward through time
        for i in range(self.N):
            prev_nodes = self.STs[-1]
            # Create new layer: up moves, middle move of last node, down move of last node
            self.ST = np.concatenate(
                (prev_nodes * self.u, 
                 [prev_nodes[-1] * self.m, 
                  prev_nodes[-1] * self.d]))
            self.STs.append(self.ST)
    
    def traverse_tree(self, payoffs):
        """ Traverse the trinomial tree backwards from expiration to present """
        # Work backwards from expiration (time N) to present (time 0)
        for i in reversed(range(self.N)):
            # Calculate option values at each node using trinomial recombination
            # payoffs[:-2] = up nodes, payoffs[1:-1] = middle nodes, payoffs[2:] = down nodes
            payoffs = (payoffs[:-2] * self.qu + 
                       payoffs[1:-1] * self.qm + 
                       payoffs[2:] * self.qd) * self.df
            
            # Check for early exercise if this is an American option
            if not self.is_european:
                payoffs = self.__check_early_exercise__(payoffs, i)
        
        return payoffs

In [35]:
""" Price an option by the binomial CRR lattice """
class BinomialCRRLattice(BinomialCRROption):
    def setup_parameters(self):
        """ Setup parameters for CRR lattice model """
        # Call parent class setup method to initialize base parameters
        super(BinomialCRRLattice, self).setup_parameters()
        # Total number of nodes in the lattice (2N+1 for N time steps)
        self.M = 2 * self.N + 1
    
    def initialize_stock_price_tree(self):
        """ Initialize stock prices for all nodes in the lattice """
        # Create array to hold all stock prices in the lattice
        self.STs = np.zeros(self.M)
        # Start from the top node (maximum number of up moves)
        self.STs[0] = self.S0 * self.u**self.N
        
        # Fill remaining nodes by moving down the lattice
        for i in range(self.M)[1:]:
            self.STs[i] = self.STs[i-1] * self.d
    
    def initialize_payoffs_tree(self):
        """ Calculate payoffs at terminal nodes (expiration) """
        # Only odd-indexed nodes are reachable at expiration in CRR lattice
        odd_nodes = self.STs[::2]
        # Calculate option payoffs based on option type
        return np.maximum(
            0, 
            (odd_nodes - self.K) if self.is_call else (self.K - odd_nodes))
    
    def __check_early_exercise__(self, payoffs, node):
        """ Check for early exercise opportunity (American options) """
        # Shorten the ends of the list to match current time step
        self.STs = self.STs[1:-1]
        # Get stock prices at odd positions (reachable nodes)
        odd_STs = self.STs[::2]
        
        # Calculate intrinsic values for early exercise
        early_ex_payoffs = \
            (odd_STs - self.K) if self.is_call \
            else (self.K - odd_STs)
        
        # Take maximum of continuation value and early exercise value
        payoffs = np.maximum(payoffs, early_ex_payoffs)
        return payoffs

In [36]:
class TrinomialLattice(TrinomialTreeOption):
    def setup_parameters(self):
        """ Setup parameters for trinomial lattice model """
        # Call parent class setup method to initialize base trinomial parameters
        super(TrinomialLattice, self).setup_parameters()
        # Total number of nodes in the lattice (2N+1 for N time steps)
        self.M = 2 * self.N + 1
    
    def initialize_stock_price_tree(self):
        """ Initialize stock prices for all nodes in the trinomial lattice """
        # Create array to hold all stock prices in the lattice
        self.STs = np.zeros(self.M)
        # Start from the top node (maximum number of up moves)
        self.STs[0] = self.S0 * self.u**self.N
        
        # Fill remaining nodes by moving down the lattice
        for i in range(self.M)[1:]:
            self.STs[i] = self.STs[i-1] * self.d
    
    def initialize_payoffs_tree(self):
        """ Calculate payoffs at terminal nodes (expiration) for trinomial lattice """
        # In trinomial lattice, all nodes are reachable at expiration
        # Calculate option payoffs based on option type
        return np.maximum(
            0, 
            (self.STs - self.K) if self.is_call else (self.K - self.STs))
    
    def __check_early_exercise__(self, payoffs, node):
        """ Check for early exercise opportunity (American options) in trinomial lattice """
        # Shorten the ends of the list to match current time step
        self.STs = self.STs[1:-1]
        
        # Calculate intrinsic values for early exercise at current nodes
        early_ex_payoffs = \
            (self.STs - self.K) if self.is_call \
            else (self.K - self.STs)
        
        # Take maximum of continuation value and early exercise value
        payoffs = np.maximum(payoffs, early_ex_payoffs)
        return payoffs